In [12]:
"""
STEP 1 (MVP-Teil):
- Graph (JSON) einlesen
- Messwerte (CSV-Ordner; Dateien heißen wie Node-IDs) einlesen
- Orientation: Verbraucherzähpfeilsystem (Fluss weg von der Sammelschiene = positiv)
- Einfache Sensitivitätsanalyse: Für jeden UW-Feeder (Kante ab Sammelschiene) wird der gemessene Feederstrom I_f(t)
  per Ridge-Regression auf die (downstream) Knotennetzleistungen P_b(t) regrediert:
      I_f(t) ≈ α_f + Σ_b S[f,b] · P_b(t)
  → S liefert eine (empirische) Sensitivität in A/kW (oder in den Einheiten der Eingangsdaten).

Annahmen (MVP):
- Netz ist radial oder zumindest die Downstream-Zuordnung je Feeder ist eindeutig (für Regression verwenden wir ohnehin nur die topologisch
  zugeordneten Knoten pro Feeder). Falls Vermaschungen existieren, kann das Verfahren dennoch laufen, die Topologie-Maske dient nur als Feature-Filter.
- CSV-Format: Spalten: timestamp, P (Leistung, Vorzeichen gemäß Verbraucherzähpfeilsystem: Einspeisung in Netz positiv)
- Zeitachsen: Wir schneiden auf den Schnitt der Zeitstempel aller benötigten Serien.
- Graph-JSON: {"nodes":[{"id":"S","type":"substation"},...], "edges":[{"id":"L1","from":"S","to":"A"}, ...]}

Ausgabe:
- sensitivities.csv: Matrix S (Zeilen: Feeder-IDs = Edge-IDs ab Substation; Spalten: Node-IDs downstream)
- report.txt: kurze Metriken (R² je Feeder, Anzahl verwendeter Knoten, n_samples)
"""
"""
STEP 1 (iterativ & auskunftsorientiert):
- Ziel: Alle nötigen Informationen AUSGEBEN, um eine erste Sensitivitätstabelle zu bestimmen.
- Vorgehen:
  1) Graph laden und Feeder + Downstream-Knoten ermitteln.
  2) Prüfen, welche Messdateien vorhanden sind (Nodes & Feeder), welche fehlen.
  3) Zeitachsen-Überlappung je Feeder (Schnittmenge) und Stichprobengröße ausgeben.
  4) Falls ausreichend Daten vorhanden: einfache Ridge-Regression je Feeder durchführen und Sensitivitätstabelle schreiben.

Konfiguration unten anpassen (Pfade, Spaltennamen, Schwellenwerte).
Ausgaben im Terminal sind so strukturiert, dass du sofort siehst, wo etwas fehlt.
"""
from __future__ import annotations
import json
from pathlib import Path
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.linear_model import Ridge

# ----------------------------
# Konfiguration
# ----------------------------
GRAPH_JSON = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\graph\graph_with_jubo_e01.json")
MEAS_DIR    = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\handling_graph\out\nodes")   # CSVs, je Datei: <node_id>.csv mit Spalten [timestamp, P]
VAL_COL = "P_Datapoint_ID"                     # Spaltenname in CSVs
TS_COL      = "timestamp"
ALPHA_RIDGE = 1.0                     # kleine Regularisierung für Stabilität
ROOT_ID     = None                    # falls None: wird aus node.type=="substation" gelesen
OUT_DIR     = Path("out_step1")
OUT_DIR.mkdir(parents=True, exist_ok=True)
MIN_SAMPLES = 200
# ----------------------------
# ----------------------------





### MVP for reading and checking for plausibolity

In [23]:

# ----------------------------
# Graph laden (Multi-Root)
# ----------------------------

def load_graph(path: Path):
    def coerce_nodes_edges(data):
        if isinstance(data, dict):
            for k in ("graph", "data", "payload", "content"):
                if k in data and isinstance(data[k], (dict, list)):
                    data = data[k]
                    break
        if isinstance(data, list) and data and isinstance(data[0], dict) and "data" in data[0]:
            nodes, edges = [], []
            for el in data:
                d = el.get("data", {})
                if "source" in d and "target" in d:
                    edges.append({
                        "id": d.get("id"),
                        "source": d.get("source"),
                        "target": d.get("target"),
                        "label": d.get("label"),
                        "features": d.get("features", {})
                    })
                elif "id" in d:
                    nodes.append({
                        "id": d.get("id"),
                        "label": d.get("label", d.get("id")),
                        "type": el.get("type") or d.get("type"),
                        "features": d.get("features", {}),
                        "position": el.get("position", {})
                    })
            return nodes, edges
        if isinstance(data, dict) and "nodes" in data and ("edges" in data or "links" in data):
            nodes = data["nodes"]
            edges = data.get("edges", data.get("links", []))
            return nodes, edges
        if isinstance(data, list) and data and isinstance(data[0], dict):
            sample = data[0]
            if ("from" in sample and "to" in sample) or ("source" in sample and "target" in sample):
                edges = data
                node_ids = {e.get("from", e.get("source")) for e in edges} | {e.get("to", e.get("target")) for e in edges}
                nodes = [{"id": n} for n in sorted(node_ids)]
                return nodes, edges
        raise ValueError("Unbekanntes JSON-Format für Graph.")

    with path.open("r", encoding="utf-8") as f:
        raw = json.load(f)
    nodes_raw, edges_raw = coerce_nodes_edges(raw)

    G = nx.Graph()
    edges_rows = []
    for n in nodes_raw:
        nid = n.get("id")
        if nid is None:
            raise ValueError("Knoten ohne 'id'.")
        attrs = {k: v for k, v in n.items() if k != "id"}
        G.add_node(str(nid), **attrs)
    for e in edges_raw:
        u = e.get("from", e.get("source"))
        v = e.get("to", e.get("target"))
        if not u or not v:
            raise ValueError("Kante ohne 'from'/'to'.")
        eid = e.get("id") or f"{u}->{v}"
        G.add_edge(str(u), str(v), id=str(eid))
        edges_rows.append({"edge_id": str(eid), "from": str(u), "to": str(v)})

    # Multi-Root Orientierung: alle Sammelschienen gleichzeitig
    roots = [n for n, a in G.nodes(data=True) if a.get("type") == "busbar"]
    if not roots:
        raise ValueError("Keine Sammelschienen (type='busbar') im Graph gefunden.")

    depths = {}
    for r in roots:
        d = nx.single_source_shortest_path_length(G, r)
        for k, v in d.items():
            depths.setdefault(k, v)

    oriented = []
    for e in edges_rows:
        u, v = e["from"], e["to"]
        du, dv = depths.get(u, np.inf), depths.get(v, np.inf)
        if G.nodes[u].get("type") == "busbar":
            frm, to = u, v
        elif G.nodes[v].get("type") == "busbar":
            frm, to = v, u
        else:
            frm, to = (u, v) if du < dv else (v, u)
        oriented.append({"edge_id": e["edge_id"], "from": frm, "to": to})

    oriented_df = pd.DataFrame(oriented)
    return G, roots, oriented_df

# ----------------------------
# Hilfsfunktionen
# ----------------------------

def feeders_from_busbars(oriented_edges: pd.DataFrame, roots: list[str]) -> pd.DataFrame:
    return oriented_edges.loc[oriented_edges["from"].isin(roots), ["edge_id", "from", "to"]].reset_index(drop=True)

def downstream_sets(oriented_edges: pd.DataFrame) -> dict[str, set]:
    Di = nx.DiGraph()
    for _, r in oriented_edges.iterrows():
        Di.add_edge(r["from"], r["to"], edge_id=r["edge_id"])
    children = {}
    for u, v in Di.edges():
        children.setdefault(u, []).append(v)

    def collect(start):
        stack = [start]
        seen = set()
        while stack:
            x = stack.pop()
            if x in seen:
                continue
            seen.add(x)
            stack.extend(children.get(x, []))
        return seen

    ds = {}
    for _, r in oriented_edges.iterrows():
        ds[r["edge_id"]] = collect(r["to"])
    return ds

def load_series(meas_dir: Path, series_id: str) -> pd.Series | None:
    f = meas_dir / f"{series_id}.csv"
    if not f.exists():
        return None

    df = pd.read_csv(f)
    if TS_COL not in df.columns or VAL_COL not in df.columns:
        raise ValueError(f"{f} muss Spalten '{TS_COL}' und '{VAL_COL}' enthalten.")

    # --- Hauptunterschied: Zeitzone vereinheitlichen ---
    # 1) immer als UTC parsen (funktioniert auch bei "+00:00"-Timestamps)
    df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce", utc=True)

    # 2) Spalte als Float; Index ist UTC-naiv (Zeitzonen entfernt)
    s = (
        df.dropna(subset=[TS_COL, VAL_COL])
          .set_index(TS_COL)[VAL_COL]
          .astype("float64")
          .sort_index()
          .tz_convert(None)   # entfernt Zeitzoneninfo (alle naiv, aber in UTC-Zeit)
    )

    return s


def fit_ridge(y: pd.Series, X: pd.DataFrame, alpha: float) -> tuple[pd.Series, float]:
    idx = y.index.intersection(X.index)
    y, X = y.loc[idx], X.loc[idx]
    if len(idx) < MIN_SAMPLES:
        return pd.Series(dtype=float), float("nan")
    Xm, Xs = X.mean(0), X.std(0).replace(0, 1.0)
    ym, ys = y.mean(), y.std()
    Xz, yz = (X - Xm) / Xs, (y - ym) / (ys if ys != 0 else 1.0)
    mdl = Ridge(alpha=alpha, fit_intercept=True)
    mdl.fit(Xz, yz)
    beta = (ys * mdl.coef_) / Xs.values
    r2 = mdl.score(Xz, yz) if ys != 0 else 1.0
    return pd.Series(beta, index=X.columns), float(r2)

# ----------------------------
# Virtuelles Feeder-Target aus UW-Feld-Zeitreihen bilden
# ----------------------------

def build_virtual_feeder_target(fid: str, ds_map: dict[str,set], meas_dir: Path, G: nx.Graph,
                                 only_uw_fields: bool = True) -> pd.Series | None:
    """
    Bildet ein virtuelles Target für den Feeder fid als SUMME der Downstream-UW-Feld-Zeitreihen.
    - Nur Knoten mit vorhandener CSV werden berücksichtigt.
    - Wenn only_uw_fields=True: es werden nur Nodes mit type=='uw_field' summiert.
    Vorzeichen: +1 pro Downstream-Knoten (Verbraucherzähpfeilrichtung).
    """
    nodes_ds = list(ds_map[fid])
    if only_uw_fields:
        nodes_ds = [n for n in nodes_ds if str(G.nodes[n].get("type")) == "uw_field"]
    parts = []
    for n in nodes_ds:
        s = load_series(meas_dir, n)
        if s is not None:
            parts.append(s.rename(n))
    if not parts:
        return None
    # Zeitlich schneiden (inner join) und aufsummieren
    X = pd.concat(parts, axis=1, join="inner").dropna()
    if X.empty:
        return None
    y = X.sum(axis=1)
    y.name = fid
    return y

# ----------------------------
# Main Pipeline
# ----------------------------
if __name__ == "__main__":
    print(" === DATA DISCOVERY (Multi-Root) ===")
    G, ROOTS, EDGES = load_graph(GRAPH_JSON)
    FEEDERS = feeders_from_busbars(EDGES, ROOTS)
    DS = downstream_sets(EDGES)
    node_ids = sorted(G.nodes())
    feeder_ids = FEEDERS["edge_id"].tolist()

    print(f"Sammelschienen erkannt: {ROOTS}")
    print(f"Knoten gesamt: {len(node_ids)} | Kanten: {EDGES.shape[0]} | Feeder: {len(feeder_ids)}")

    print(" -- Verfügbare Messdateien (Nodes) --")
    have_nodes = [n for n in node_ids if (MEAS_DIR / f"{n}.csv").exists()]
    missing_nodes = [n for n in node_ids if n not in have_nodes]
    print(f"vorhanden: {len(have_nodes)} | fehlend: {len(missing_nodes)}")

    print(" -- Verfügbare Messdateien (Feeder) -- (virtuell aus Downstream-Summe)")
    print("Hinweis: Feeder-Targets werden als Summe der Downstream-UW-Felder gebildet.")

    print(" === ZEITACHSE & FEATURE-SELEKTION JE FEEDER ===")
    rows_diag, sensitivities = [], []
    for fid in feeder_ids:
        # 1) Virtuelles Target (Summe der Downstream-UW-Felder)
        y = build_virtual_feeder_target(fid, DS, MEAS_DIR, G, only_uw_fields=True)
        if y is None:
            print(f"Feeder {fid}: kein virtuelles Target (keine Downstream-UW-Feld-Zeitreihen) → übersprungen")
            rows_diag.append({"feeder": fid, "n_samples": 0, "features": 0, "r2": np.nan})
            continue

        # 2) Prädiktoren = einzelne Downstream-UW-Feld-Zeitreihen (die im Target stecken)
        ds_nodes = [n for n in DS[fid] if n in have_nodes and str(G.nodes[n].get("type")) == "uw_field"]
        X_parts = [load_series(MEAS_DIR, n).rename(n) for n in ds_nodes if load_series(MEAS_DIR, n) is not None]
        if not X_parts:
            print(f"Feeder {fid}: keine Downstream-Knoten mit Messdaten → übersprungen")
            rows_diag.append({"feeder": fid, "n_samples": 0, "features": 0, "r2": np.nan})
            continue
        X = pd.concat(X_parts, axis=1, join="inner").dropna()
        # 3) Schnittmenge der Zeitachsen zwischen y und X
        idx = y.index.intersection(X.index)
        n_samples = len(idx)
        print(f"Feeder {fid}: Downstream-Features={len(X.columns)} | gemeinsame Samples={n_samples}")
        if n_samples < MIN_SAMPLES:
            print(f"  → zu wenig Samples (<{MIN_SAMPLES}); Regression wird übersprungen")
            rows_diag.append({"feeder": fid, "n_samples": n_samples, "features": len(X.columns), "r2": np.nan})
            continue

        # 4) Regression (liefert Sensitivitäten ~1.0, dient aber als Konsistenz-Check und für spätere Erweiterungen)
        beta, r2 = fit_ridge(y.loc[idx], X.loc[idx], ALPHA_RIDGE)
        rows_diag.append({"feeder": fid, "n_samples": n_samples, "features": len(X.columns), "r2": r2})
        sensitivities.append(beta.rename(fid))

    diag_df = pd.DataFrame(rows_diag)
    diag_path = OUT_DIR / "diagnostics.csv"
    diag_df.to_csv(diag_path, index=False)
    print(" === DIAGNOSTIK ===")
    print(diag_df.to_string(index=False))
    print(f"→ gespeichert: {diag_path}")

    if sensitivities:
        S = pd.DataFrame(sensitivities).fillna(0.0)
        S_path = OUT_DIR / "sensitivities.csv"
        S.to_csv(S_path)
        print(" === ERSTE SENSITIVITÄTEN (virtuell) ===")
        print(S.head().to_string())
        print(f"→ gespeichert: {S_path}")
    else:
        print(" Keine ausreichenden Daten für eine Sensitivitätstabelle. Siehe diagnostics.csv für Details.")


 === DATA DISCOVERY (Multi-Root) ===
Sammelschienen erkannt: ['SHUW', 'Tarp', 'JUBO']
Knoten gesamt: 11 | Kanten: 10 | Feeder: 5
 -- Verfügbare Messdateien (Nodes) --
vorhanden: 6 | fehlend: 5
 -- Verfügbare Messdateien (Feeder) -- (virtuell aus Downstream-Summe)
Hinweis: Feeder-Targets werden als Summe der Downstream-UW-Felder gebildet.
 === ZEITACHSE & FEATURE-SELEKTION JE FEEDER ===
Feeder SHUW_SS_E24: Downstream-Features=3 | gemeinsame Samples=1343
Feeder TARP_SS_TARP_E01: Downstream-Features=1 | gemeinsame Samples=1343
Feeder JUBO_E03_JUBO: Downstream-Features=1 | gemeinsame Samples=1344
Feeder JUBO_E02_JUBO: Downstream-Features=1 | gemeinsame Samples=1344
Feeder JUBO_E01_JUBO: Downstream-Features=1 | gemeinsame Samples=1344
 === DIAGNOSTIK ===
          feeder  n_samples  features       r2
     SHUW_SS_E24       1343         3 0.999997
TARP_SS_TARP_E01       1343         1 0.999999
   JUBO_E03_JUBO       1344         1 0.999999
   JUBO_E02_JUBO       1344         1 0.999999
   JU

## Creating a first actual sensitivity analysis

In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from pathlib import Path
import json
import networkx as nx

GRAPH_JSON = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\graph\graph_with_jubo_e01.json")
MEAS_DIR   = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\handling_graph\out\nodes")
TS_COL     = "timestamp"
VAL_COL    = "P_Datapoint_ID"
ALPHA_RIDGE = 0.5
MIN_SAMPLES = 200

# --- 1. UW-Felder extrahieren ---
def load_uw_fields(graph_path: Path):
    data = json.loads(graph_path.read_text(encoding="utf-8"))
    uw_nodes = []
    for el in data:
        d = el.get("data", {})
        if d.get("type") == "uw_field":
            uw_nodes.append(d["id"])
    return sorted(uw_nodes)

uw_nodes = load_uw_fields(GRAPH_JSON)
print(f"Gefundene UW-Felder: {uw_nodes}")

# --- 2. Messwerte laden und auf gemeinsame Zeitachse bringen ---
def load_series(name):
    df = pd.read_csv(MEAS_DIR / f"{name}.csv")
    df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce", utc=True)
    s = df.dropna(subset=[TS_COL, VAL_COL]).set_index(TS_COL)[VAL_COL].astype(float)
    return s.tz_convert(None).sort_index()

series = {n: load_series(n) for n in uw_nodes if (MEAS_DIR / f"{n}.csv").exists()}

df_all = pd.concat(series.values(), axis=1, join="inner")
df_all.columns = list(series.keys())
print(f"Gemeinsame Zeitpunkte: {len(df_all)} | UW-Felder im Datensatz: {df_all.shape[1]}")

if len(df_all) < MIN_SAMPLES:
    raise RuntimeError("Zu wenige gemeinsame Zeitpunkte für robuste Regression.")

# --- 3. Ridge-Regression je UW-Feld ---
S = pd.DataFrame(index=df_all.columns, columns=df_all.columns, dtype=float)
r2_scores = {}

for target in df_all.columns:
    y = df_all[target]
    X = df_all.drop(columns=[target])
    X = (X - X.mean()) / X.std().replace(0, 1)
    y = (y - y.mean()) / y.std()
    mdl = Ridge(alpha=ALPHA_RIDGE)
    mdl.fit(X, y)
    beta = pd.Series(mdl.coef_, index=X.columns)
    S.loc[target, X.columns] = beta
    r2_scores[target] = mdl.score(X, y)

print("\n=== SENSITIVITÄTSMATRIX UW↔UW ===")
print(S.round(3))
print("\nR² je Regressionsziel:")
for k,v in r2_scores.items():
    print(f"{k}: {v:.4f}")

S.to_csv("out_step2_sensitivity_matrix.csv")
print("\n→ gespeichert: out_step2_sensitivity_matrix.csv")


Gefundene UW-Felder: ['JUBO_E01', 'JUBO_E02', 'JUBO_E03', 'SHUW_E24', 'Tarp_E01']
Gemeinsame Zeitpunkte: 1343 | UW-Felder im Datensatz: 5

=== SENSITIVITÄTSMATRIX UW↔UW ===
          JUBO_E01  JUBO_E02  JUBO_E03  SHUW_E24  Tarp_E01
JUBO_E01       NaN    -0.790    -0.356    -0.001     0.000
JUBO_E02    -1.261       NaN    -0.448     0.001    -0.001
JUBO_E03    -2.795    -2.205       NaN    -0.002     0.000
SHUW_E24    -0.345     0.507    -0.158       NaN     0.001
Tarp_E01     0.311    -0.413     0.045     0.001       NaN

R² je Regressionsziel:
JUBO_E01: 1.0000
JUBO_E02: 1.0000
JUBO_E03: 1.0000
SHUW_E24: 0.5835
Tarp_E01: 0.4756

→ gespeichert: out_step2_sensitivity_matrix.csv


# BOLS fehlt hier

# Weitere Steps (Vorläufig:)

In [30]:
# Voraussetzung: df_all existiert bereits

import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

# 1) Differenzieren + Standardisieren
df_d = df_all.diff().dropna()
# sehr kleine Varianzen droppen
stds = df_d.std()
df_d = df_d.drop(columns=stds[stds < 1e-9].index)
df_z = (df_d - df_d.mean()) / df_d.std().replace(0, 1)

# 2) Lagwahl kleingehalten (robuster)
maxlags = 1
try:
    sel = VAR(df_z).select_order(maxlags=maxlags)
    p = int(np.clip(sel.aic, 1, 2))   # p∈{1,2}
except Exception:
    p = 1
print(f"VAR mit p={p}")
var = VAR(df_z).fit(maxlags=p)

# 3) IRFs OHNE Cholesky (direkt aus A_l), d.h. nicht-orthogonalisiert
#    y_t = sum_{l=1..p} A_l y_{t-l} + u_t, A_l ∈ R^{k×k}
A_list = [var.coefs[l] for l in range(p)]  # shape: (p, k, k)

def irfs_from_A(A_list, H: int):
    """Nicht-orthogonalisierte IRFs Ψ_0..Ψ_H via Rekursion: Ψ_h = sum_l A_l Ψ_{h-l}."""
    k = A_list[0].shape[0]
    Psis = [np.eye(k)]
    for h in range(1, H+1):
        acc = np.zeros((k, k))
        for l, A_l in enumerate(A_list, start=1):
            if h - l >= 0:
                acc += A_l @ Psis[h - l]
        Psis.append(acc)
    return Psis  # Liste: Ψ_0, Ψ_1, ..., Ψ_H (je k×k)

H = 1
Psis = irfs_from_A(A_list, H=H)
M_total = np.sum(Psis, axis=0)  # aggregierte Einwirkungsmatrix bis Horizont H

uw_order = list(df_z.columns)
M_df = pd.DataFrame(M_total, index=uw_order, columns=uw_order)
print("\nAggregierte Einwirkungsmatrix (nicht-orthogonalisierte IRFs):")
print(M_df.round(3))

# Hilfsfunktion: Reaktionsvektor für Impuls in UW A (Δp über alle UWs pro 1 MW in A)
def reaction_vector(uw_name, delta=1.0):
    a_idx = uw_order.index(uw_name)
    v = M_total[:, a_idx] * delta
    return pd.Series(v, index=uw_order)


VAR mit p=1

Aggregierte Einwirkungsmatrix (nicht-orthogonalisierte IRFs):
          JUBO_E01  JUBO_E02  JUBO_E03  SHUW_E24  Tarp_E01
JUBO_E01     1.027    -0.052     0.042    -0.142     0.178
JUBO_E02    -0.006     1.018    -0.023     0.105    -0.134
JUBO_E03    -0.055     0.090     0.946     0.126    -0.154
SHUW_E24     0.002     0.024    -0.057     1.075    -0.100
Tarp_E01     0.006     0.017    -0.050    -0.166     1.184


c:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [33]:
# === PTDF (x=1), Impuls->Flüsse, historische Flüsse & deterministische Bänder ===
import json
import numpy as np
import pandas as pd
import networkx as nx
from pathlib import Path

# -------- 1) PTDF je verbundener Komponente (Slack = Busbar falls vorhanden) --------
def build_ptdf_components(graph_json_path: Path):
    elems = json.loads(Path(graph_json_path).read_text(encoding="utf-8"))
    G = nx.Graph()
    for el in elems:
        d = el.get("data", {})
        if 'source' in d and 'target' in d:
            G.add_edge(d['source'], d['target'], x=1.0)  # Einheitsreaktanz
        elif d.get("id") and d.get("type") in {"busbar", "uw_field"}:
            G.add_node(d["id"], ntype=d.get("type"))

    comps = []
    for nodes in nx.connected_components(G):
        Gc = G.subgraph(nodes).copy()
        # Knoten & Indizes
        nodes = list(Gc.nodes())
        idx = {n:i for i,n in enumerate(nodes)}
        edges = list(Gc.edges())
        m, n = len(edges), len(nodes)
        if n == 0 or m == 0:
            continue

        # Inzidenz A (m x n); Orientierung beliebig, aber konsistent
        A = np.zeros((m, n))
        for e_i,(u,v) in enumerate(edges):
            A[e_i, idx[u]] = +1.0
            A[e_i, idx[v]] = -1.0

        # Slack: bevorzugt Busbar, sonst 1. Knoten
        busbars = [n for n in nodes if Gc.nodes[n].get('ntype') == 'busbar']
        slack_node = busbars[0] if busbars else nodes[0]
        slack_idx = idx[slack_node]

        keep = [i for i in range(n) if i != slack_idx]
        B = A.T @ A  # x=1 => Laplacian
        B_rr = B[np.ix_(keep, keep)]
        # Pseudoinverse ist robust, auch wenn B_rr nicht vollen Rang hat
        B_rr_inv = np.linalg.pinv(B_rr)

        # PTDF: f = A * theta; theta_r = B_rr^{-1} * p_r  -> f = A[:,keep] * B_rr^{-1} * p_r
        H = A[:, keep] @ B_rr_inv  # (m x (n-1))

        comps.append({
            "graph": Gc,
            "nodes": nodes,
            "edges": edges,
            "A": A,
            "PTDF": H,
            "keep": keep,
            "slack_idx": slack_idx,
            "slack_node": slack_node
        })
    return comps

ptdf_components = build_ptdf_components(GRAPH_JSON)
print(f"PTDF für {len(ptdf_components)} Komponente(n) aufgebaut.")
if not ptdf_components:
    raise RuntimeError("Keine PTDF-Komponente gefunden (Graph leer?).")

# -------- 2) Reaktionsvektor aus M_total (Impuls in A -> Δp aller UWs) --------
def reaction_vector(uw_name: str, delta: float = 1.0) -> pd.Series:
    a = uw_order.index(uw_name)
    v = M_total[:, a] * delta
    return pd.Series(v, index=uw_order)

# -------- 3) Impuls -> Linienflussänderungen (über alle Komponenten) --------
def flows_from_impulse(uw_name: str, delta: float = 1.0) -> pd.Series:
    v = reaction_vector(uw_name, delta=delta)  # Δp pro UW
    all_flows = []
    for comp in ptdf_components:
        nodes = comp["nodes"]; keep = comp["keep"]; Hm = comp["PTDF"]; edges = comp["edges"]; sidx = comp["slack_idx"]
        # Δp nur auf Knoten der Komponente abbilden
        p_full = np.zeros(len(nodes))
        for i, n in enumerate(nodes):
            if n in v.index:
                p_full[i] = v[n]
        # Slack balancieren (Summe null)
        p_full[sidx] = - (p_full.sum() - p_full[sidx])
        pr = p_full[keep]
        f = Hm @ pr  # (m,)
        edge_names = [f"e:{u}--{v}" for (u,v) in edges]
        all_flows.append(pd.Series(f, index=edge_names))
    if all_flows:
        out = pd.concat(all_flows)
        # sinnvolle Sortierung nach Betrag
        return out.reindex(out.abs().sort_values(ascending=False).index)
    return pd.Series(dtype=float)

# Quick sanity: Top-Kanten für einen Beispielimpuls
try:
    print("\nTop-10 Fluss-Änderungen bei +1 MW in JUBO_E01:")
    print(flows_from_impulse("JUBO_E01", 1.0).head(10).round(4))
except Exception as e:
    print("Hinweis:", e)

# -------- 4) Historische Linienflüsse aus ΔP(t) für Quantil-Grenzen --------
def compute_historical_flows(df_delta: pd.DataFrame) -> pd.DataFrame:
    all_frames = []
    for comp in ptdf_components:
        nodes = comp["nodes"]; keep = comp["keep"]; Hm = comp["PTDF"]; edges = comp["edges"]; sidx = comp["slack_idx"]
        X = df_delta.reindex(columns=nodes, fill_value=0.0).copy()
        P = X.values  # (T x n)
        # Slack-Spalte balancieren
        P[:, sidx] = - (P.sum(axis=1) - P[:, sidx])
        P_r = P[:, keep]                   # (T x (n-1))
        F = (P_r @ Hm.T)                   # (T x m) => f = H * p_r  => F = P_r * H^T
        cols = [f"e:{u}--{v}" for (u,v) in edges]
        all_frames.append(pd.DataFrame(F, index=X.index, columns=cols))
    return pd.concat(all_frames, axis=1)

df_delta = df_all.diff().dropna()
F_hist = compute_historical_flows(df_delta)
print(f"\nHistorische Flüsse: {F_hist.shape[0]} Zeitpunkte, {F_hist.shape[1]} Leitungen")

# Grenzwerte konservativ über hohes Quantil
alpha_quant = 0.995
Fmax = F_hist.abs().quantile(alpha_quant, axis=0)   # Serie je Leitung
# --- ab hier den alten Band-Teil ersetzen ---

# Grenzwerte-Basis (Median bleibt)
f0 = F_hist.median(axis=0)

# 1) Robuste Limits & aktive Leitungen
k_sigma = 4.0
eps_cap = 1e-3  # ggf. erhöhen (z.B. 0.01)
Fstd = F_hist.std(axis=0)

Fmax_robust = pd.concat([
    F_hist.abs().quantile(0.995, axis=0),
    k_sigma * Fstd,
    pd.Series(eps_cap, index=F_hist.columns)
], axis=1).max(axis=1)

active_edges = Fmax_robust[Fmax_robust > eps_cap].index.tolist()
print(f"Aktive Leitungen für Bandberechnung: {len(active_edges)} / {F_hist.shape[1]}")

# 2) Bandfunktion (nur aktive Kanten, robustes Limit)
def deterministic_band_for_UW_robust(uw_name: str) -> tuple[float, float]:
    f_unit = flows_from_impulse(uw_name, 1.0)
    if f_unit.empty:
        return 0.0, 0.0
    f_unit = f_unit.reindex(active_edges).fillna(0.0)
    if (f_unit.abs() <= 1e-12).all():
        return 0.0, 0.0

    f0_act   = f0.reindex(active_edges).fillna(0.0)
    Fcap_act = Fmax_robust.reindex(active_edges).fillna(eps_cap)

    ratios = []
    for e in active_edges:
        denom = abs(f_unit[e])
        if denom <= 1e-12:
            continue
        cap = max(Fcap_act[e] - abs(f0_act[e]), 0.0)
        ratios.append(cap / denom)

    if not ratios:
        return 0.0, 0.0
    band = float(np.min(ratios))
    return band, band  # symmetrisch

bands_rob = {uw: deterministic_band_for_UW_robust(uw) for uw in uw_order}
bands_rob_df = pd.DataFrame(bands_rob, index=["DeltaP_plus_MW", "DeltaP_minus_MW"]).T
print("\n=== Deterministische Leistungsbänder (robust) ===")
print(bands_rob_df.round(3))



PTDF für 1 Komponente(n) aufgebaut.

Top-10 Fluss-Änderungen bei +1 MW in JUBO_E01:
e:SHUW--SHUW_E24      -0.9755
e:SHUW_E24--JUBO_A5   -0.9730
e:JUBO_E01--JUBO_A5    0.9668
e:JUBO_E01--JUBO       0.0605
e:JUBO_E03--JUBO      -0.0550
e:JUBO_A5--BOLS_A5    -0.0062
e:Tarp_E01--BOLS_A5    0.0062
e:JUBO_E02--JUBO      -0.0055
e:Tarp--Tarp_E01      -0.0000
e:BOLS_A5--BOLS_E42    0.0000
dtype: float64

Historische Flüsse: 1342 Zeitpunkte, 10 Leitungen
Aktive Leitungen für Bandberechnung: 7 / 10

=== Deterministische Leistungsbänder (robust) ===
          DeltaP_plus_MW  DeltaP_minus_MW
JUBO_E01          15.753           15.753
JUBO_E02           9.555            9.555
JUBO_E03           5.262            5.262
SHUW_E24          39.508           39.508
Tarp_E01          12.951           12.951


In [34]:
# === Stochastische Bänder (Chance-Constraints) ==================================
from math import isfinite

# 1) Unsicherheitsmaß pro Leitung
Fstd = F_hist.std(axis=0).reindex(active_edges).fillna(0.0)

def stochastic_band_for_UW(uw_name: str, alpha: float) -> tuple[float, float, list]:
    """Chance-Constraint-Band: zieht z*σ_e von der Kapazität je Kante ab.
       Rückgabe: (Band+, Band-, list(bindende_kanten))"""
    z = {0.05: 1.645, 0.01: 2.326}.get(alpha, None)
    if z is None:
        raise ValueError("alpha nur 0.05 oder 0.01 im Beispiel.")
    f_unit = flows_from_impulse(uw_name, 1.0).reindex(active_edges).fillna(0.0)
    if (f_unit.abs() <= 1e-12).all():
        return 0.0, 0.0, []

    f0_act   = f0.reindex(active_edges).fillna(0.0)
    Fcap_act = Fmax_robust.reindex(active_edges).fillna(0.0)
    Fsig_act = Fstd.reindex(active_edges).fillna(0.0)

    ratios = []
    binders = []
    for e in active_edges:
        denom = abs(f_unit[e])
        if denom <= 1e-12:
            continue
        # effektives Limit unter Unsicherheit
        cap_eff = Fcap_act[e] - z * Fsig_act[e] - abs(f0_act[e])
        cap_eff = max(cap_eff, 0.0)
        val = cap_eff / denom if denom > 0 else np.inf
        if isfinite(val):
            ratios.append((val, e))

    if not ratios:
        return 0.0, 0.0, []

    band = float(min(ratios, key=lambda t: t[0])[0])
    # bindende kanten innerhalb 1% um das Minimum
    minv = band
    binders = [e for v,e in ratios if v <= 1.01*minv]
    return band, band, binders

# 2) Bänder für alpha=5% und 1% berechnen
results_5 = {}
results_1 = {}
binders_map = {}
for uw in uw_order:
    b5p, b5m, bind5 = stochastic_band_for_UW(uw, alpha=0.05)
    b1p, b1m, bind1 = stochastic_band_for_UW(uw, alpha=0.01)
    results_5[uw] = (b5p, b5m)
    results_1[uw] = (b1p, b1m)
    binders_map[uw] = {"alpha=5%": bind5, "alpha=1%": bind1}

bands_5_df = pd.DataFrame(results_5, index=["DeltaP_plus_MW", "DeltaP_minus_MW"]).T
bands_1_df = pd.DataFrame(results_1, index=["DeltaP_plus_MW", "DeltaP_minus_MW"]).T

print("\n=== Stochastische Leistungsbänder (α = 5%) ===")
print(bands_5_df.round(3))
print("\n=== Stochastische Leistungsbänder (α = 1%) ===")
print(bands_1_df.round(3))

# 3) Zeig, welche Kanten limitieren (Binder)
print("\nBindende Kanten je UW (innerhalb 1% des Minimums):")
for uw, m in binders_map.items():
    print(f"- {uw}: {m}")



=== Stochastische Leistungsbänder (α = 5%) ===
          DeltaP_plus_MW  DeltaP_minus_MW
JUBO_E01           9.190            9.190
JUBO_E02           5.758            5.758
JUBO_E03           3.298            3.298
SHUW_E24          24.761           24.761
Tarp_E01           7.556            7.556

=== Stochastische Leistungsbänder (α = 1%) ===
          DeltaP_plus_MW  DeltaP_minus_MW
JUBO_E01           6.473            6.473
JUBO_E02           4.186            4.186
JUBO_E03           2.485            2.485
SHUW_E24          17.863           17.863
Tarp_E01           5.322            5.322

Bindende Kanten je UW (innerhalb 1% des Minimums):
- JUBO_E01: {'alpha=5%': ['e:SHUW_E24--JUBO_A5'], 'alpha=1%': ['e:SHUW_E24--JUBO_A5']}
- JUBO_E02: {'alpha=5%': ['e:JUBO_E02--JUBO'], 'alpha=1%': ['e:JUBO_E02--JUBO']}
- JUBO_E03: {'alpha=5%': ['e:JUBO_E03--JUBO'], 'alpha=1%': ['e:JUBO_E03--JUBO']}
- SHUW_E24: {'alpha=5%': ['e:JUBO_E03--JUBO'], 'alpha=1%': ['e:SHUW--SHUW_E24']}
- Tarp_E01: {'alph

In [35]:
# === Einheiten-Korrektur: z-Score <-> MW ===
# Voraussetzung: df_d = df_all.diff().dropna(); df_z = (df_d - mean)/std
sigma = df_d.std().replace(0, 1)  # Std-Abweichungen der ΔP in MW pro UW (Index = uw_order)

def reaction_vector_mw(uw_name: str, delta_mw: float = 1.0) -> pd.Series:
    """
    Reaktionsvektor in MW für einen Impuls von delta_mw (MW) in UW 'uw_name'.
    M_total ist in z-Einheiten, daher:
      Impuls in z:   delta_z_source = delta_mw / sigma[A]
      Reaktion in z: M_total[:,A] * delta_z_source
      zurück nach MW: multiply elementweise mit sigma[i]
    """
    a = uw_order.index(uw_name)
    delta_z_source = delta_mw / float(sigma.iloc[a])
    resp_z = M_total[:, a] * delta_z_source  # z-Einheiten an allen UWs
    resp_mw = pd.Series(resp_z, index=uw_order) * sigma  # zurück in MW
    return resp_mw

# flows_from_impulse soll MW liefern -> passe die Funktion an:
def flows_from_impulse(uw_name: str, delta_mw: float = 1.0) -> pd.Series:
    v = reaction_vector_mw(uw_name, delta_mw=delta_mw)  # jetzt MW
    all_flows = []
    for comp in ptdf_components:
        nodes = comp["nodes"]; keep = comp["keep"]; Hm = comp["PTDF"]; edges = comp["edges"]; sidx = comp["slack_idx"]
        p_full = np.zeros(len(nodes))
        for i, n in enumerate(nodes):
            if n in v.index:
                p_full[i] = v[n]
        # Slack-Ausgleich innerhalb der Komponente
        p_full[sidx] = - (p_full.sum() - p_full[sidx])
        pr = p_full[keep]
        f = Hm @ pr  # (m,)
        edge_names = [f"e:{u}--{v}" for (u,v) in edges]
        all_flows.append(pd.Series(f, index=edge_names))
    if all_flows:
        out = pd.concat(all_flows)
        return out.reindex(out.abs().sort_values(ascending=False).index)
    return pd.Series(dtype=float)

# Optional: kurzer Check - 1 MW-Impuls sollte Flüsse in "vernünftiger" Größenordnung geben
print("Sanity (1 MW in JUBO_E01):")
print(flows_from_impulse("JUBO_E01", 1.0).head(5).round(4))


Sanity (1 MW in JUBO_E01):
e:SHUW--SHUW_E24      -1.0158
e:SHUW_E24--JUBO_A5   -1.0088
e:JUBO_E01--JUBO_A5    1.0000
e:JUBO_E01--JUBO       0.0273
e:JUBO_E03--JUBO      -0.0226
dtype: float64


In [36]:
# === TPTDF (A->B) und Paar-Bänder =============================================

def locate_component_for_nodes(A_name: str, B_name: str):
    """Finde die PTDF-Komponente, in der A und B gemeinsam liegen."""
    for comp in ptdf_components:
        nodes = comp["nodes"]
        if (A_name in nodes) and (B_name in nodes):
            return comp
    raise ValueError(f"A={A_name} und B={B_name} liegen nicht in derselben Netzkomponente.")

def tptdf_vector(A_name: str, B_name: str, comp) -> pd.Series:
    """1 MW Transfer A->B (Slack-frei, slack-invariant): liefert Fluss je Leitung in der Komponente."""
    nodes = comp["nodes"]; keep = comp["keep"]; Hm = comp["PTDF"]; edges = comp["edges"]
    a = nodes.index(A_name); b = nodes.index(B_name)
    pr = np.zeros(len(keep))
    for i, k in enumerate(keep):
        if k == a: pr[i] = +1.0
        elif k == b: pr[i] = -1.0
    f = Hm @ pr
    return pd.Series(f, index=[f"e:{u}--{v}" for (u,v) in edges])

def pair_transfer_limit(A_name: str, B_name: str, alpha: float = 0.05) -> tuple[float, list]:
    """
    Maximaler sicherer Transfer (MW) von A nach B unter Chance-Constraint (alpha).
    Nutzt die robusten Limits + z*σ_e-Abschlag aus deiner stochastischen Band-Logik.
    Rückgabe: (MW_limit, bindende_kanten)
    """
    comp = locate_component_for_nodes(A_name, B_name)
    f_unit = tptdf_vector(A_name, B_name, comp)  # Fluss je 1 MW Transfer

    # Nur aktive Kanten betrachten (vereinheitlicht mit deiner Bandlogik)
    f_unit = f_unit.reindex(active_edges).fillna(0.0)

    # Effektive Grenzen: Fcap_eff = Fmax_robust - z * σ_e - |f0|
    z = {0.05: 1.645, 0.01: 2.326}.get(alpha, 1.645)
    Fsig_act = F_hist.std(axis=0).reindex(active_edges).fillna(0.0)
    Fcap_eff = (Fmax_robust - z * Fsig_act - f0.abs()).reindex(active_edges).fillna(0.0)
    Fcap_eff = Fcap_eff.clip(lower=0.0)

    ratios = []
    for e in active_edges:
        denom = abs(f_unit.get(e, 0.0))
        if denom <= 1e-12:
            continue
        ratios.append((Fcap_eff[e] / denom, e))
    if not ratios:
        return 0.0, []

    limit = float(min(ratios, key=lambda t: t[0])[0])
    binders = [e for v,e in ratios if v <= 1.01*limit]  # binnen 1% am Limit
    return limit, binders

# Beispiel: zulässiger Transfer JUBO_E01 -> SHUW_E24
lim_5, bind_5 = pair_transfer_limit("JUBO_E01", "SHUW_E24", alpha=0.05)
lim_1, bind_1 = pair_transfer_limit("JUBO_E01", "SHUW_E24", alpha=0.01)
print(f"\nMax sicherer Transfer JUBO_E01 -> SHUW_E24:  α=5%: {lim_5:.2f} MW  |  α=1%: {lim_1:.2f} MW")
print("Bindende Kanten (5%):", bind_5)
print("Bindende Kanten (1%):", bind_1)



Max sicherer Transfer JUBO_E01 -> SHUW_E24:  α=5%: 8.94 MW  |  α=1%: 6.30 MW
Bindende Kanten (5%): ['e:SHUW_E24--JUBO_A5']
Bindende Kanten (1%): ['e:SHUW_E24--JUBO_A5']


# Neue Idee

In [ ]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from pathlib import Path

# --------------------- Pfade & Konstanten ---------------------
GRAPH_JSON = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\graph\graph_with_jubo_e01.json")
MEAS_DIR   = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\handling_graph\out\nodes")
TS_COL     = "timestamp"
VAL_COL    = "P_Datapoint_ID"
MIN_SAMPLES = 200

# Fixe Annahmen für Stromberechnung
U_KV_DEFAULT = 110.0   # Leitungsebene
COS_PHI      = 0.95

# --------------------- Graph laden & Limits korrekt auslesen ---------------------
elems = json.loads(GRAPH_JSON.read_text(encoding="utf-8"))

G = nx.Graph()
node_meta = {}
edge_meta = {}

for el in elems:
    d = el.get("data", {})
    f = d.get("features", {})  # <--- NEU: features-Block einbeziehen

    # 1) Edges (mit source/target)
    if "source" in d and "target" in d:
        u, v = d["source"], d["target"]
        edge_id = d.get("id", f"{u}__{v}")
        limit_A = f.get("Strom_Limit_in_A") or d.get("Strom_Limit_in_A")  # erst aus features, dann fallback
        try:
            limit_A = float(limit_A) if limit_A is not None else None
        except Exception:
            limit_A = None

        G.add_edge(u, v, id=edge_id, x=1.0, Strom_Limit_in_A=limit_A)
        edge_meta[edge_id] = {"u": u, "v": v, "Strom_Limit_in_A": limit_A}

        # Sicherheitshalber Nodes anlegen
        if u not in G.nodes:
            G.add_node(u)
        if v not in G.nodes:
            G.add_node(v)

    # 2) Nodes (uw_field, battery, etc.)
    elif "id" in d:
        nid = d["id"]
        ntype = d.get("type")
        limit_A = f.get("Strom_Limit_in_A") or d.get("Strom_Limit_in_A")
        try:
            limit_A = float(limit_A) if limit_A is not None else None
        except Exception:
            limit_A = None

        G.add_node(nid, ntype=ntype, Strom_Limit_in_A=limit_A)
        node_meta[nid] = {"type": ntype, "Strom_Limit_in_A": limit_A}

# --------------------- Messknoten bestimmen (uw_field + battery) ---------------------
meas_nodes = [nid for nid, meta in node_meta.items() if meta.get("type") in {"uw_field", "battery"}]
print(f"Messknoten (uw_field/battery): {sorted(meas_nodes)}")

# --------------------- Zeitreihen laden (LEVELS in MW) ---------------------
def load_series(name: str) -> pd.Series:
    p = MEAS_DIR / f"{name}.csv"
    if not p.exists():
        return None
    df = pd.read_csv(p)
    df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce", utc=True)
    s = df.dropna(subset=[TS_COL, VAL_COL]).set_index(TS_COL)[VAL_COL].astype(float)
    return s.tz_convert(None).sort_index()

series = {}
for n in meas_nodes:
    s = load_series(n)
    if s is not None:
        series[n] = s

if not series:
    raise RuntimeError("Keine Messreihen gefunden.")

df_all = pd.concat(series.values(), axis=1, join="inner")
df_all.columns = list(series.keys())
print(f"Gemeinsame Zeitpunkte: {len(df_all)} | Messknoten im Datensatz: {df_all.shape[1]}")
if len(df_all) < MIN_SAMPLES:
    print("Warnung: Wenige gemeinsame Zeitpunkte. Prüfe Datenabdeckung.")

# --------------------- PTDF je Komponente (x=1, Slack = Busbar falls vorhanden) ---------------------
def build_ptdf_components(G: nx.Graph):
    comps = []
    for nodes in nx.connected_components(G):
        Gc = G.subgraph(nodes).copy()
        nodes = list(Gc.nodes())
        edges = list(Gc.edges())
        if not nodes or not edges:
            continue
        idx = {n:i for i,n in enumerate(nodes)}
        m, n = len(edges), len(nodes)
        # Inzidenz
        A = np.zeros((m, n))
        edge_ids = []
        for e_i, (u, v) in enumerate(edges):
            A[e_i, idx[u]] = +1.0
            A[e_i, idx[v]] = -1.0
            edge_ids.append(Gc[u][v].get("id", f"{u}__{v}"))
        # Slack
        busbars = [n for n in nodes if Gc.nodes[n].get("ntype") == "busbar"]
        slack_node = busbars[0] if busbars else nodes[0]
        slack_idx = idx[slack_node]
        keep = [i for i in range(n) if i != slack_idx]
        # PTDF
        B = A.T @ A
        B_rr = B[np.ix_(keep, keep)]
        H = A[:, keep] @ np.linalg.pinv(B_rr)
        comps.append({
            "graph": Gc,
            "nodes": nodes,
            "edges": edges,
            "edge_ids": edge_ids,        
            "A": A,
            "PTDF": H,
            "keep": keep,
            "slack_idx": slack_idx,
            "slack_node": slack_node
        })
    return comps

ptdf_components = build_ptdf_components(G)
print(f"PTDF für {len(ptdf_components)} Komponente(n) aufgebaut.")
if not ptdf_components:
    raise RuntimeError("Keine PTDF-Komponente berechnet.")

# --------------------- Flüsse aus LEVELS P(t) (MW) ---------------------
def compute_flows_from_levels(df_levels: pd.DataFrame) -> pd.DataFrame:
    frames = []
    for comp in ptdf_components:
        nodes = comp["nodes"]; keep = comp["keep"]; Hm = comp["PTDF"]; sidx = comp["slack_idx"]
        colmap = {c:i for i,c in enumerate(nodes)}
        # baue P(t) für diese Komponente
        X = df_levels.reindex(columns=nodes, fill_value=0.0).copy()
        P = X.values  # (T x n)
        # Slack-Bilanz je Zeitstempel
        P[:, sidx] = - (P.sum(axis=1) - P[:, sidx])
        P_r = P[:, keep]         # (T x (n-1))
        F = (P_r @ Hm.T)         # (T x m)
        cols = comp["edge_ids"]  # echte Edge-IDs
        frames.append(pd.DataFrame(F, index=X.index, columns=cols))
    return pd.concat(frames, axis=1)

F_hist_levels = compute_flows_from_levels(df_all)   # gesamte Historie auf Flüsse (MW)

# --------------------- Strom & Auslastung am letzten Timestamp ---------------------
def mw_to_ampere(P_MW: pd.Series, U_kV: float = U_KV_DEFAULT, cos_phi: float = COS_PHI) -> pd.Series:
    # I[A] = P[MW]*1e6 / (sqrt(3)*U[V]*cosφ) = P[MW]*1e3 / (√3*U[kV]*cosφ)
    denom = (np.sqrt(3.0) * U_kV * cos_phi)
    return (P_MW * 1e3) / denom

last_ts = df_all.index.max()

# a) Edge-Report: Flüsse (MW) -> Ströme (A) -> Auslastung
F_last = F_hist_levels.loc[last_ts]  # Serie je Edge-ID in MW
I_edge = mw_to_ampere(F_last.abs())  # Betrag in A
# Limits je Edge mappen
edge_limits = pd.Series({eid: edge_meta.get(eid, {}).get("Strom_Limit_in_A") for eid in F_last.index})
edge_util = I_edge / edge_limits
edge_rows = pd.DataFrame({
    "element_id": F_last.index,
    "type": "edge",
    "timestamp": last_ts,
    "P_MW": F_last.values,
    "U_kV": U_KV_DEFAULT,
    "I_A": I_edge.values,
    "I_limit_A": edge_limits.values,
    "utilization": edge_util.values
})

# b) Node-Report (uw_field + battery + optional andere Node-Typen mit Limits)
#    P_MW = Einspeisung am Knoten (positiv/negativ je Messung), Strom daraus
node_last = df_all.loc[last_ts].reindex(list(node_meta.keys())).fillna(0.0)
node_types = pd.Series({nid: node_meta[nid].get("type") for nid in node_meta})
node_limits = pd.Series({nid: node_meta[nid].get("Strom_Limit_in_A") for nid in node_meta})
# nur Knoten mit Messwerten (meas_nodes), sonst P=0
node_last = node_last.reindex(node_types.index).fillna(0.0)

I_node = mw_to_ampere(node_last.abs())
node_util = I_node / node_limits
node_rows = pd.DataFrame({
    "element_id": node_last.index,
    "type": node_types.values,
    "timestamp": last_ts,
    "P_MW": node_last.values,
    "U_kV": U_KV_DEFAULT,
    "I_A": I_node.values,
    "I_limit_A": node_limits.values,
    "utilization": node_util.values
})

# c) Kombinierter Report (mit klarer Bezeichnung)
edge_rows = edge_rows.copy()
node_rows = node_rows.copy()

# --- zusätzliche Spalten für Klarheit ---
# für Edges: Start- und Zielknoten ergänzen (aus edge_meta)
edge_rows["from_node"] = edge_rows["element_id"].map(lambda eid: edge_meta.get(eid, {}).get("u"))
edge_rows["to_node"]   = edge_rows["element_id"].map(lambda eid: edge_meta.get(eid, {}).get("v"))

# für Nodes: Typ direkt anzeigen (z. B. uw_field, battery)
node_rows["from_node"] = None
node_rows["to_node"]   = None

# --- kombinieren ---
report = pd.concat([edge_rows, node_rows], axis=0, ignore_index=True)

# Numerik bereinigen
def _to_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

report["I_limit_A"] = report["I_limit_A"].map(_to_float)
report["I_A"] = report["I_A"].astype(float)
report["utilization"] = report["I_A"] / report["I_limit_A"]
# --- Herkunft des Limits (limit_source) ergänzen ---
def find_limit_source(row):
    if row["type"] == "edge":
        meta = edge_meta.get(row["element_id"], {})
        if meta.get("Strom_Limit_in_A") is not None:
            return row["element_id"]
        # Falls Edge kein Limit hat, aber einer der Endknoten schon:
        u, v = meta.get("u"), meta.get("v")
        if u in node_meta and node_meta[u].get("Strom_Limit_in_A"):
            return u
        if v in node_meta and node_meta[v].get("Strom_Limit_in_A"):
            return v
        return None
    else:
        # Für Nodes: eigene ID, falls Limit existiert
        if node_meta.get(row["element_id"], {}).get("Strom_Limit_in_A"):
            return row["element_id"]
        return None

report["limit_source"] = report.apply(find_limit_source, axis=1)



# --- Sortieren ---
report_sorted = report.sort_values(["utilization"], ascending=[False], na_position="last")

# --- Ausgabe ---
cols_show = [
    "element_id", "type", "from_node", "to_node",
    "timestamp", "P_MW", "U_kV", "I_A", "I_limit_A", "utilization"
]

print(f"\n=== Auslastung zum letzten Zeitpunkt ({last_ts}) ===")
print(report_sorted[cols_show].head(25).round(3))

# --- Export ---
out_path = GRAPH_JSON.parent / "auslastung_last_timestamp.csv"
report_sorted[cols_show].to_csv(out_path, index=False)
print(f"\n→ Exportiert: {out_path}")

# Ausgabe
print(f"\n=== Auslastung zum letzten Zeitpunkt ({last_ts}) ===")
print(report_sorted.head(25).round(3))

# Export
out_path = GRAPH_JSON.parent / "auslastung_last_timestamp.csv"
report_sorted.to_csv(out_path, index=False)
print(f"\n→ Exportiert: {out_path}")


Messknoten (uw_field/battery): ['BOLS_E42', 'JUBO_E01', 'JUBO_E02', 'JUBO_E03', 'SHUW_E24', 'Tarp_E01']
Gemeinsame Zeitpunkte: 1343 | Messknoten im Datensatz: 6
PTDF für 1 Komponente(n) aufgebaut.

=== Auslastung zum letzten Zeitpunkt (2025-10-21 05:00:00) ===
                                 element_id      type from_node   to_node  \
3   110_SHUW_TARP_GELB_JUBO_JUBO_A5_BOLS_A5      edge   BOLS_A5   JUBO_A5   
7      110_SHUW_TARP_GELB_JUBO_BOLS_A5_TARP      edge  Tarp_E01   BOLS_A5   
18                                 Tarp_E01  uw_field      None      None   
11                                 SHUW_E24  uw_field      None      None   
6      110_SHUW_TARP_GELB_JUBO_BOLS_A5_BOLS      edge   BOLS_A5  BOLS_E42   
16                                 BOLS_E42   battery      None      None   
1      110_SHUW_TARP_GELB_JUBO_SHUW_JUBO_A5      edge  SHUW_E24   JUBO_A5   
2      110_SHUW_TARP_GELB_JUBO_JUBO_A5_JUBO      edge   JUBO_A5  JUBO_E01   
0                               SHUW_SS_E24   

# Mit more printing of things

In [ ]:
import json
import numpy as np
import networkx as nx
from pathlib import Path

# ============================================================
# 0) PFAD & EINSTELLUNGEN
# ============================================================
GRAPH_JSON = Path(r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\graph\graph_with_jubo_e01.json")

# ============================================================
# 1) GRAPH LADEN UND AUFBAUEN
# ============================================================
elems = json.loads(GRAPH_JSON.read_text(encoding="utf-8"))
G = nx.Graph()

for el in elems:
    d = el.get("data", {})
    f = d.get("features", {}) or {}

    # --- Edges ---
    if "source" in d and "target" in d:
        u, v = d["source"], d["target"]
        # Reaktanz (falls vorhanden), sonst 1.0
        x_val = f.get("reactance") or f.get("X") or d.get("reactance") or 1.0
        try:
            x_val = float(x_val)
        except Exception:
            x_val = 1.0

        G.add_edge(u, v, x=x_val, id=d.get("id", f"{u}__{v}"))

        # Sicherheitshalber Knoten hinzufügen
        if u not in G.nodes:
            G.add_node(u, ntype=None)
        if v not in G.nodes:
            G.add_node(v, ntype=None)

    # --- Nodes ---
    elif "id" in d:
        nid = d["id"]
        ntype = d.get("type")
        G.add_node(nid, ntype=ntype)

print("=== 1) KNOTEN UND LEITUNGEN ===")
print("Knoten:", list(G.nodes()))
print("Leitungen:")
for e in G.edges(data=True):
    print(" ", e)
print()

# ============================================================
# 2) INDEXE FESTLEGEN
# ============================================================
nodes = list(G.nodes())
edges = list(G.edges())
n = len(nodes)
m = len(edges)
idx = {nname: i for i, nname in enumerate(nodes)}

print("=== 2) INDEXE ===")
print("Knoten-Index:", idx)
print(f"Anzahl Knoten = {n}, Anzahl Leitungen = {m}")
print()

# ============================================================
# 3) INZIDENZMATRIX A AUFBAUEN
# ============================================================
A = np.zeros((m, n))
x_list = []
edge_ids = []

for e_i, (u, v) in enumerate(edges):
    A[e_i, idx[u]] = +1.0   # willkürliche Richtung u -> v
    A[e_i, idx[v]] = -1.0
    x_val = G[u][v].get("x", 1.0)
    try:
        x_val = float(x_val)
    except Exception:
        x_val = 1.0
    x_list.append(x_val)
    edge_ids.append(G[u][v].get("id", f"{u}__{v}"))

print("=== 3) INZIDENZMATRIX A (m x n) ===")
print(A)
print("Reaktanzen je Leitung (x):", x_list)
print("Edge-IDs:", edge_ids)
print()

# ============================================================
# 4) SLACK-KNOTEN BESTIMMEN
# ============================================================
busbars = [nn for nn in nodes if G.nodes[nn].get("ntype") == "busbar"]
if busbars:
    slack_node = busbars[0]
else:
    slack_node = nodes[0]
slack_idx = idx[slack_node]

print("=== 4) SLACK ===")
print("Slack-Knoten:", slack_node, "(Index:", slack_idx, ")")
print()

# ============================================================
# 5) REDUZIERTE INZIDENZ A_r (SPALTE DES SLACK RAUS)
# ============================================================
keep = [i for i in range(n) if i != slack_idx]
A_r = A[:, keep]

print("=== 5) REDUZIERTE INZIDENZ A_r ===")
print(A_r)
print("Behaltene Knoten:", [nodes[i] for i in keep])
print()

# ============================================================
# 6) DIAGONALE X^{-1} (ADMINTTANZEN)
# ============================================================
X_inv = np.diag([1.0 / xv for xv in x_list])

print("=== 6) X^{-1} (Diagonalmatrix der Leitungsadmittanzen) ===")
print(X_inv)
print()

# ============================================================
# 7) REDUZIERTE BUS-MATRIX B_rr
# ============================================================
B_rr = A_r.T @ X_inv @ A_r

print("=== 7) REDUZIERTE BUS-MATRIX B_rr ===")
print(B_rr)
print()

# ============================================================
# 8) PSEUDOINVERSE B_rr_inv
# ============================================================
B_rr_inv = np.linalg.pinv(B_rr)

print("=== 8) PSEUDOINVERSE VON B_rr ===")
print(B_rr_inv)
print()

# ============================================================
# 9) PTDF BERECHNEN
# ============================================================
PTDF = X_inv @ A_r @ B_rr_inv

print("=== 9) PTDF-MATRIX ===")
print("Zeilen = Leitungen:", edge_ids)
print("Spalten = Knoten (ohne Slack):", [nodes[i] for i in keep])
print(PTDF)
print()

# ============================================================
# 10) TEST-INJEKTION
# ============================================================
test_node_idx = keep[0]
test_node_name = nodes[test_node_idx]

deltaP_r = np.zeros(len(keep))
deltaP_r[0] = 1.0  # +1 MW Einspeisung am ersten Nicht-Slack-Knoten

deltaF = PTDF @ deltaP_r

print("=== 10) TEST-INJEKTION ===")
print(f"+1 MW am Knoten '{test_node_name}' (Slack '{slack_node}' nimmt -1 MW auf)")
print("→ resultierende Flussänderungen je Leitung (in MW):")
for e_name, f_val in zip(edge_ids, deltaF):
    print(f"  {e_name:35s}  {f_val: .5f}")
print()


=== 1) KNOTEN UND LEITUNGEN ===
Knoten: ['SHUW', 'SHUW_E24', 'JUBO_A5', 'JUBO_E01', 'JUBO_E03', 'BOLS_A5', 'BOLS_E42', 'Tarp', 'Tarp_E01', 'JUBO_E02', 'JUBO']
Leitungen:
  ('SHUW', 'SHUW_E24', {'x': 1.0, 'id': 'SHUW_SS_E24'})
  ('SHUW_E24', 'JUBO_A5', {'x': 1.0, 'id': '110_SHUW_TARP_GELB_JUBO_SHUW_JUBO_A5'})
  ('JUBO_A5', 'JUBO_E01', {'x': 1.0, 'id': '110_SHUW_TARP_GELB_JUBO_JUBO_A5_JUBO'})
  ('JUBO_A5', 'BOLS_A5', {'x': 1.0, 'id': '110_SHUW_TARP_GELB_JUBO_JUBO_A5_BOLS_A5'})
  ('JUBO_E01', 'JUBO', {'x': 1.0, 'id': 'JUBO_E01_JUBO'})
  ('JUBO_E03', 'JUBO', {'x': 1.0, 'id': 'JUBO_E03_JUBO'})
  ('BOLS_A5', 'BOLS_E42', {'x': 1.0, 'id': '110_SHUW_TARP_GELB_JUBO_BOLS_A5_BOLS'})
  ('BOLS_A5', 'Tarp_E01', {'x': 1.0, 'id': '110_SHUW_TARP_GELB_JUBO_BOLS_A5_TARP'})
  ('Tarp', 'Tarp_E01', {'x': 1.0, 'id': 'TARP_SS_TARP_E01'})
  ('JUBO_E02', 'JUBO', {'x': 1.0, 'id': 'JUBO_E02_JUBO'})

=== 2) INDEXE ===
Knoten-Index: {'SHUW': 0, 'SHUW_E24': 1, 'JUBO_A5': 2, 'JUBO_E01': 3, 'JUBO_E03': 4, 'BOLS_A5': 5,